In [1]:
import subprocess
import yaml
from time import sleep
import os
import numpy as np

In [2]:
def ticcmd(*args):
    return subprocess.check_output(['ticcmd'] + list(args))

def get_position():
    status = yaml.load(ticcmd('-s', '--full'))
    position = status['Current position']
    return position

def set_position(target_position):
    position = get_position()
    ticcmd('--resume', '--position', str(target_position))
    while position != target_position:
        position = get_position()
        print(position, end="\r")
        sleep(0.1)
    #sleep(0.5)
    #print(position)
    
def set_cavity_frequency(current_frequency, target_frequency, calibration_constant):
    frequency_change = target_frequency - current_frequency
    steps_required = int(np.rint(frequency_change / calibration_constant))
    target_position = get_position() + steps_required
    set_position(target_position)
    
def reset_position(initial_position):
    set_position(initial_position - 500)
    set_position(initial_position)

In [12]:
initial_frequency = 2
initial_position = get_position()
c_a = 75472.30159
c_b = -660762.15316
c_c = 1934011.96347

frequencies = [initial_frequency, 2.6, 2.7, 2.8, 2.9]
counts = [0] * len(frequencies)
number_of_runs = 1
integration_time = 0.02
counter_frequency = 1/integration_time
sleep_time = 0.5

In [15]:
smiq.cw_on()
counterlogic.set_count_length(300)
counterlogic.set_count_frequency(counter_frequency)
counterlogic.startCount()

0

In [16]:
for i in range(number_of_runs):
    for i in range(len(frequencies)):
        if i == 0:
            target_frequency = frequencies[i]
            current_frequency = frequencies[i]
            smiq.set_cw(frequency=target_frequency)
            sleep(sleep_time)
            counts[i] = counts[i] + counterlogic.countdata[0,-1]

        if i == len(frequencies):
            target_frequency = frequencies[i]
            current_frequency = frequencies[i - 1]
            set_cavity_frequency(current_frequency, target_frequency, calibration_constant)
            smiq.set_cw(frequency=target_frequency)
            sleep(sleep_time)
            counts[i] = counts[i] + counterlogic.countdata[0,-1]
            reset_position(initial_position)

        else:
            target_frequency = frequencies[i]
            current_frequency = frequencies[i - 1]
            set_cavity_frequency(current_frequency, target_frequency, calibration_constant)
            smiq.set_cw(frequency=target_frequency)
            smiq.cw_on()
            sleep(sleep_time)
            counts[i] = counts[i] + counterlogic.countdata[0,-1]
            smiq.off()

2.6 2.6
2.7 2.6
2.8 2.7
2.9 2.8


In [17]:
#Sets current position to 0
zero_position = 0
ticcmd('--exit-safe-start', '--halt-and-set-position', str(zero_position))

#Get current count parameters
counterlogic.get_count_length()
counterlogic.get_count_frequency()

# Brute force way to control anything, pull gui levers.
counter._mw.start_counter_Action.trigger()

b''

In [5]:
def set_cavity_frequency(offset, target_frequency, co_3, co_2, co_1, co_0):
    frequency_y2_adjusted = int(np.rint(co_3*target_frequency**3 + co_2*target_frequency**2 + co_1*target_frequency + co_0 - offset))
    set_position(frequency_y2_adjusted)

In [18]:
initial_frequency = 2.687314
initial_position = -5000
target_frequency = 2.78

co_3 = 69278.68
co_2 = -609102.93
co_1 = 1790504.24
co_0 = - 1762367.85

frequency_y1 = int(np.rint(co_3*initial_frequency**3 + co_2*initial_frequency**2 + co_1*initial_frequency + co_0))
offset = frequency_y1 - initial_position

In [19]:
set_cavity_frequency(offset, target_frequency, co_3, co_2, co_1, co_0)

In [17]:
reset_position(initial_position)